In [ ]:
!pip install --q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import streamlit as st
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import os

# Load pre-trained MobileNetV2 model with custom layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom classification layers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(38, activation='softmax')(x)  # Assuming 38 disease classes
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Save model
model.save("plant_disease_model.h5")

# Load model for inference
model = load_model("plant_disease_model.h5")

# Streamlit UI
disease_dict = {
    0: ("Apple___Apple_scab", "To cure apple scab, use fungicides like captan or chlorothalonil early in the season for prevention. If the disease is present, apply fungicides such as myclobutanil or tebuconazole. Prune infected leaves, fruit, and twigs to improve air circulation and reduce spore spread. Dispose of fallen infected material and consider using mulch to prevent spores from splashing back onto the tree.."),
    1: ("Apple___Black_rot", "remove and destroy any infected leaves, fruit, and branches to prevent further spread of the disease. Use copper-based fungicides or myclobutanil to treat infected areas. It is also essential to maintain good air circulation around the tree and practice proper pruning techniques. Infected fruit should be removed from the tree promptly, and fallen fruit should be discarded to reduce the chance of reinfection. Additionally, applying a fungicide spray at regular intervals during the growing season will help manage the disease."),
    2: ("Apple___Cedar_apple_rust", "emove and destroy any infected leaves and fruit to reduce the spread of the disease. Pruning and thinning the tree to improve air circulation can also help reduce the conditions favorable for rust growth. Applying fungicides, such as chlorothalonil or myclobutanil, during the growing season, especially during early spring when the disease is most active, can prevent new infections. Planting rust-resistant apple varieties is another preventive measure. It is also advised to remove nearby cedar trees, which are the alternate hosts of the fungus. Regular monitoring and treating infected areas early will help manage cedar apple rust."),
    # 3: ("Apple___healthy", "No disease detected. Maintain plant health."),
    # 4: ("Blueberry___healthy", "No disease detected. Keep monitoring."),
    5: ("Cherry_(including_sour)___Powdery_mildew", "start by removing and destroying any infected plant parts, such as leaves and stems, to reduce the spread of the disease. Applying fungicides such as sulfur-based or potassium bicarbonate products can effectively control the fungus. Ensure proper air circulation around the plant by thinning out dense areas and avoiding overcrowding, as powdery mildew thrives in humid and stagnant conditions. Additionally, watering the plants at the base rather than overhead can help reduce moisture on the leaves, which is a key factor in the development of powdery mildew. Regular monitoring and early intervention are essential for effective control."),
    # 6: ("Cherry_(including_sour)___healthy", "No disease detected."),
    7: ("Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot", "To treat **Cercospora Leaf Spot** and **Gray Leaf Spot** on corn (maize), apply fungicides that are effective against these fungal infections. It's important to rotate crops regularly to reduce the buildup of the disease in the soil. In addition, remove and destroy infected plant debris after harvest to minimize the risk of reinfection. Implementing proper irrigation techniques, such as drip irrigation, can help keep the leaves dry, as the disease thrives in wet conditions. Regular monitoring of the crop is essential to catch early signs and apply treatments promptly."),
    8: ("Corn_(maize)___Common_rust_", "Use resistant hybrids and fungicides."),
    9: ("Corn_(maize)___Northern_Leaf_Blight", "To treat **Northern Leaf Blight** on corn (maize), it is recommended to use fungicides specifically targeted for this disease. Timing is crucial, so applying fungicides at the right growth stage—usually during the early to mid-flowering period—can be effective. Crop rotation is another key strategy to reduce the recurrence of the disease, as it helps break the disease cycle. Additionally, removing infected leaves and plant debris from the field after harvest can help prevent the fungus from overwintering. Using resistant corn varieties can also provide long-term protection against Northern Leaf Blight. Regular monitoring of the crop is essential for early detection and effective management."),
    # 10: ("Corn_(maize)___healthy", "No disease detected."),
    11: ("Grape___Black_rot", "To treat **Grape Black Rot**, it's important to implement a combination of cultural and chemical practices. Pruning affected vines and removing infected plant material can help reduce the spread of the disease. Applying fungicides at the right stages of vine growth, particularly during the early stages of infection, is crucial in controlling the spread. It’s also important to ensure proper vine spacing to improve air circulation and reduce humidity around the plants, as this can help minimize the environment conducive to fungal growth. Regularly monitoring the vineyard for signs of infection and applying fungicides as necessary can significantly reduce the impact of Black Rot on grape crops."),
    12: ("Grape___Esca_(Black_Measles)", "Remove infected vines and avoid injuries."),
    13: ("Grape___Leaf_blight_(Isariopsis_Leaf_Spot)", "Apply fungicides and proper pruning."),
    # 14: ("Grape___healthy", "No disease detected."),
    15: ("Orange___Haunglongbing_(Citrus_greening)", "Remove infected trees and control psyllids."),
    16: ("Peach___Bacterial_spot", "Use copper-based sprays and remove infected leaves."),
    # 17: ("Peach___healthy", "No disease detected."),
    18: ("Pepper,_bell___Bacterial_spot", "Avoid overhead watering and apply copper sprays."),
    # 19: ("Pepper,_bell___healthy", "No disease detected."),
    20: ("Potato___Early_blight", "Use certified seeds and apply fungicides."),
    21: ("Potato___Late_blight", "Practice crop rotation and destroy infected plants."),
    # 22: ("Potato___healthy", "No disease detected."),
    # 23: ("Raspberry___healthy", "No disease detected."),
    # 24: ("Soybean___healthy", "No disease detected."),
    25: ("Squash___Powdery_mildew", "Apply neem oil or sulfur sprays."),
    26: ("Strawberry___Leaf_scorch", "Remove infected leaves and apply fungicides."),
    # 27: ("Strawberry___healthy", "No disease detected."),
    28: ("Tomato___Bacterial_spot", "Use copper-based sprays."),
    29: ("Tomato___Early_blight", "Use resistant varieties and remove infected leaves."),
    30: ("Tomato___Late_blight", "Destroy infected plants and use fungicides."),
    31: ("Tomato___Leaf_Mold", "To treat **Leaf Mold** on tomatoes, improve air circulation around the plants by spacing them adequately and pruning any excess foliage. This reduces humidity around the plant, which helps prevent the mold from spreading. Remove and dispose of any affected leaves to reduce the risk of further infection. Avoid overhead watering to prevent water from staying on the leaves. Fungicides that are labeled for use against fungal leaf diseases can also be applied to control the spread of the infection. Ensuring proper ventilation and maintaining dry foliage is key to preventing leaf mold"),
    32: ("Tomato___Septoria_leaf_spot", "Apply fungicides and avoid wet leaves."),
    33: ("Tomato___Spider_mites Two-spotted_spider_mite", "To treat **Two-Spotted Spider Mites** on tomatoes or other plants, begin by using insecticidal soap or horticultural oils to kill the mites on contact. You can also spray the plants with a strong stream of water to dislodge the mites from the plant leaves. Reducing dust and keeping the environment humid can make conditions less favorable for the mites. Additionally, introducing natural predators, such as ladybugs or predatory mites, can help control the spider mite population. Repeating treatments may be necessary, as spider mites can quickly build resistance. It's important to regularly monitor your plants to detect the presence of mites early before they cause extensive damage."),
    34: ("Tomato___Target_Spot", "Remove infected foliage and apply fungicides."),
    35: ("Tomato___Tomato_Yellow_Leaf_Curl_Virus", "To cure **Tomato Yellow Leaf Curl Virus (TYLCV)**, there is no direct cure once the plant is infected. The best course of action is to remove and destroy any infected plants to prevent the spread of the virus. The virus is primarily spread by whiteflies, so controlling the whitefly population is crucial. Use insecticidal soaps or neem oil to manage whiteflies, and consider using yellow sticky traps to capture them. To prevent future infections, plant virus-resistant tomato varieties and ensure proper plant spacing to reduce stress and increase airflow. Additionally, practicing crop rotation and removing plant debris can help reduce the likelihood of virus transmission.."),
    36: ("Tomato___Tomato_mosaic_virus", "remove and destroy any infected plants immediately to prevent the virus from spreading to healthy ones. Disinfect all gardening tools, stakes, and supports used for the infected plants with a bleach solution or a suitable disinfectant. Avoid handling healthy plants after touching infected ones to prevent cross-contamination. It's crucial to control the vector, such as aphids, which can spread the virus. Using resistant tomato varieties is another preventive measure to reduce the impact of this disease. Always ensure to buy virus-free seedlings from reliable sources. Unfortunately, there is no cure once the plants are infected, so early detection and removal are key to managing the virus."),
    # 37: ("Tomato___healthy", "No disease detected.")
}


def predict_disease(image):
    image = cv2.resize(image, (128, 128))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    class_index = np.argmax(prediction)
    return class_index, prediction

st.sidebar.image(['/content/n1 - Copy.png', '/content/n2 - Copy.png', '/content/n3 (1).png'], use_container_width=True)

# Increase heading size and change color to green
st.markdown(""" <h1 style='text-align: center; color: green; font-size: 36px;'>🌿PLANT DISEASE MANAGEMENT SYSTEM </h1> """, unsafe_allow_html=True)

# User guidelines
st.markdown("""<h2 style='color: black;'>User Guidelines:</h2>
<ul>
<li>Only upload clear images of the <b>diseased part</b> of the plant.</li>
<li>Avoid images with excessive background or multiple leaves.</li>
<li>Ensure good lighting for better accuracy in detection.</li>
</ul>
""", unsafe_allow_html=True)

uploaded_file = st.file_uploader("📤 Upload a plant leaf image", type=["jpg", "png", "jpeg"])

if uploaded_file:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    st.image(image, caption='🌿 Uploaded Image', use_container_width=True)
    class_index, confidence = predict_disease(image)
    if class_index in disease_dict:
        disease_name, cure = disease_dict[class_index]
    else:
        closest_class = max(disease_dict.keys(), key=lambda k: confidence[0][k])
        disease_name, cure = disease_dict[closest_class]
    st.write(f"🦠 Predicted Disease:** {disease_name}")
    st.write(f"💊 Suggested Cure:** {cure}")


from transformers import pipeline

@st.cache_resource
def load_qa_pipeline():
    return pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

qa_pipeline = load_qa_pipeline()

# Generate context from the disease_dict
context = "\n".join([f"{name}: {treatment}" for _, (name, treatment) in disease_dict.items()])

st.markdown("---")
st.markdown("### 🤖 Ask a Plant Disease Question")
user_question = st.text_input("💬 Ask your question here (e.g., How to treat black rot?)")

if user_question:
    with st.spinner("Searching for the best answer..."):
        result = qa_pipeline({
            'question': user_question,
            'context': context
        })
        st.write(f"🧠 Answer:** {result['answer']}")

Writing app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

35.245.113.190


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.113.190:8501

⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://free-falcons-scream.loca.lt
2025-05-29 06:56:14.149159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748501774.175977    1129 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748501774.183899    1129 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-29 06:56:14.211260: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary i